In [ ]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math

pd.options.mode.use_inf_as_na = True

# Choose whether absolute or relative pupil size to be used

In [ ]:
absoluteSize = False # True means absolute, False means relative

In [ ]:
KeyInclude = ['IncreaseTimeDwell', 'DecreaseTimeDwell']
TimeDwellOrig = 800
TimeFixation = 300

In [ ]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [ ]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtScratchPad(UserLooksAtKey):
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstScratchPad = 0
    flagStart = 0
    
    
    # Find unique looks at scratchpad, by checking if progress pct is greater than previous one and store first look in a list
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        if row[1] == 'ScratchPad':
            
            # for consecutive scratchpad looks
            if UserLooksAtKey[nRow-1][1] == 'ScratchPad':
                if float(row[2]) == 0:
                    progressScratchPad = float(UserLooksAtKey[nRow-1][2])
                else:
                    progressScratchPad = float(row[2])
    
            # if first time of the look at scratchpad
            else:
                # if not the very first look (means the variable for progress exists)
                if firstScratchPad > 0: 
                    # if the current progress value is less than the previous one, means it is a new look 
                    if progressScratchPad > float(row[2]):
                        # if progress is not 0, then update the progress --> sometimes, it is 0 at the end of the look
                        if float(row[2]) != 0:
                            progressScratchPad = float(row[2])
                            epochList.append(row)
                            #print(row)
                            
                # if the very first look (means the variable for progress does not exist)    
                else:
                    progressScratchPad = float(row[2])
                    firstScratchPad = 1
                    epochList.append(row)
                    #print(row)
            
    #print(epochList)

    return epochList

In [ ]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [ ]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [ ]:
# function to remove all invalid data in gazelog
def cleanGazeLog(gazeLog):
    gazeLogNew = list()
    for rowInd in enumerate(gazeLog):
        if 'Invalid' not in gazeLog[rowInd[0]]:
            gazeLogNew.append(gazeLog[rowInd[0]])
    return gazeLogNew

In [ ]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    
    return(vals)

In [ ]:
def appendDictionary(dictionary, variableName, dataAbsoluteL, dataAbsoluteR, dataRelativeL, dataRelativeR, absoluteSize):
    if dictionary[variableName+'First'] == 0:
                
        dictionary[variableName+'First'] = 1
        if absoluteSize: # check if absoluteSize is set to true 
            dictionary[variableName+'Left'] = [dataAbsoluteL.values[i][0] for i in range(0, len(dataAbsoluteL.values))]
            dictionary[variableName+'Right'] = [dataAbsoluteR.values[i][0] for i in range(0, len(dataAbsoluteR.values))]
                    
            dictionary[variableName+'LeftSquared'] = [(dataAbsoluteL.values[i][0])**2 for i in range(0, len(dataAbsoluteL.values))]
            dictionary[variableName+'RightSquared'] = [(dataAbsoluteR.values[i][0])**2 for i in range(0, len(dataAbsoluteR.values))]
                    
            #print(len(dictionary[variableName+'Left']), len(dictionary[variableName+'LeftSquared']))
    
        else:
            dictionary[variableName+'Left'] = [dataRelativeL.values[i][0] for i in range(0, len(dataRelativeL.values))]
            dictionary[variableName+'Right'] = [dataRelativeR.values[i][0] for i in range(0, len(dataRelativeR.values))]

            dictionary[variableName+'LeftSquared'] = [(dataRelativeL.values[i][0])**2 for i in range(0, len(dataRelativeL.values))]
            dictionary[variableName+'RightSquared'] = [(dataRelativeR.values[i][0])**2 for i in range(0, len(dataRelativeR.values))]
                    
            #print(len(dictionary[variableName+'Left']), len(dictionary[variableName+'LeftSquared']))
    
        dictionary[variableName+'Number'] = dictionary[variableName+'Number'] + 1
    else:
        if absoluteSize:
            dataAddL = [dictionary[variableName+'Left'][i]+dataAbsoluteL.values[i][0] for i in range(0, min(len(dataAbsoluteL[dataAbsoluteL.columns[0]]), len(dictionary[variableName+'Left'])))]
            dataAddR = [dictionary[variableName+'Right'][i]+dataAbsoluteR.values[i][0] for i in range(0, min(len(dataAbsoluteR[dataAbsoluteR.columns[0]]), len(dictionary[variableName+'Right'])))]
                    
            dataAddSquareL = [dictionary[variableName+'LeftSquared'][i]+(dataAbsoluteL.values[i][0])**2 for i in range(0, min(len(dataAbsoluteL[dataAbsoluteL.columns[0]]), len(dictionary[variableName+'LeftSquared'])))]
            dataAddSquareR = [dictionary[variableName+'RightSquared'][i]+(dataAbsoluteR.values[i][0])**2 for i in range(0, min(len(dataAbsoluteR[dataAbsoluteR.columns[0]]), len(dictionary[variableName+'RightSquared'])))]
                    
        else:
            dataAddL = [dictionary[variableName+'Left'][i]+dataRelativeL.values[i][0] for i in range(0, min(len(dataRelativeL[dataRelativeL.columns[0]]), len(dictionary[variableName+'Left'])))]
            dataAddR = [dictionary[variableName+'Right'][i]+dataRelativeR.values[i][0] for i in range(0, min(len(dataRelativeR[dataRelativeR.columns[0]]), len(dictionary[variableName+'Right'])))]

            dataAddSquareL = [dictionary[variableName+'LeftSquared'][i]+(dataRelativeL.values[i][0])**2 for i in range(0, min(len(dataRelativeL[dataRelativeL.columns[0]]), len(dictionary[variableName+'LeftSquared'])))]
            dataAddSquareR = [dictionary[variableName+'RightSquared'][i]+(dataRelativeR.values[i][0])**2 for i in range(0, min(len(dataRelativeR[dataRelativeR.columns[0]]), len(dictionary[variableName+'RightSquared'])))]

        dictionary[variableName+'Left'] = dataAddL
        dictionary[variableName+'Right'] = dataAddR
        dictionary[variableName+'LeftSquared'] = dataAddSquareL
        dictionary[variableName+'RightSquared'] = dataAddSquareR
        dictionary[variableName+'Number'] = dictionary[variableName+'Number'] + 1
    
    return dictionary

In [ ]:
def computeAggregateAverage(scratchPadLookedAtEpoch, scratchPadList, phraseList, GazeLog, pupilData, TimeDwell, subjName):
    
    rowTimeList = -1
    phraseScratchedList = list()
    phraseToBeScratchedList = list()
    timeUserLooked = list()
    timeScratchAll = list()
    timeToBeScratchedAll = list()
   
    phraseScratchedInd = 0
    phraseToBeScratchedInd = 0
    
    timeStrUserLooked =  [item[0] for item in scratchPadLookedAtEpoch]
    timeStrScratch = [item1[0] for item1 in scratchPadList]
    timeStrToBeScracthed = [item2[0] for item2 in phraseList]
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    
    timeUserLooked = timeConversion(timeStrUserLooked)
    timeScratchAll = timeConversion(timeStrScratch)
    timeToBeScratchedAll = timeConversion(timeStrToBeScracthed)
    timeGazeLog = timeConversion(timeStrGazeLog)
        
    # Create list of pupil sizes from gazelog
    pupilLogL_woFilter = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in GazeLog]
    pupilLogR_woFilter = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in GazeLog]
    
    winSize = 40 # in Per's paper, he used a 13 sample window for 30Hz eye tracker
    # Filter pupil sizes
    pupilLogL = hampel(pupilLogL_woFilter, winSize, 3)
    pupilLogR = hampel(pupilLogR_woFilter, winSize, 3)

    # Pupil distance (x,y,z) in User Coordinate system 
    pupilDistLx = [float(item6[3]) if 'Invalid' not in item6 else np.nan for item6 in GazeLog]
    pupilDistLy = [float(item7[4]) if 'Invalid' not in item7 else np.nan for item7 in GazeLog]
    pupilDistLz = [float(item8[5]) if 'Invalid' not in item8 else np.nan for item8 in GazeLog]
    pupilDistRx = [float(item9[10]) if 'Invalid' not in item9 else np.nan for item9 in GazeLog]
    pupilDistRy = [float(item10[11]) if 'Invalid' not in item10 else np.nan for item10 in GazeLog]
    pupilDistRz = [float(item11[12]) if 'Invalid' not in item11 else np.nan for item11 in GazeLog]
    
    # create dataframe from the pupil distances
    df_pupilDist = pd.DataFrame(np.column_stack([pupilDistLx, pupilDistLy, pupilDistLz, pupilDistRx, pupilDistRy, pupilDistRz]), columns=['pupilDistLx', 'pupilDistLy', 'pupilDistLz', 'pupilDistRx', 'pupilDistRy', 'pupilDistRz'])
    df_pupilDist = df_pupilDist.interpolate('linear', limit_area = 'inside')
    #print(df_pupilDist.pupilDistLx)
    
    pupilDist = [math.sqrt((df_pupilDist.pupilDistLx[indPt]-df_pupilDist.pupilDistRx[indPt])**2 + (df_pupilDist.pupilDistLy[indPt]-df_pupilDist.pupilDistRy[indPt])**2 + (df_pupilDist.pupilDistLz[indPt]-df_pupilDist.pupilDistRz[indPt])**2) for indPt in range(0, len(pupilDistLx))]
    
    #print(timeUserLooked)
    
    for timeList in timeUserLooked:
        rowTimeList = rowTimeList + 1
        # Extract epoch data from gazelog
        
        timeScratchPadActive = timeList - datetime.timedelta(milliseconds=float(scratchPadLookedAtEpoch[rowTimeList][3])) - datetime.timedelta(milliseconds=TimeFixation)
        timeGazeLogStart, GazeLogStartInd = nearestTimePoint(timeGazeLog, timeScratchPadActive)
        
        timeWindowEnd = timeList + datetime.timedelta(seconds=5)
        
        timeGazeLogEnd, GazeLogEndInd = nearestTimePoint(timeGazeLog, timeWindowEnd)
        #print(GazeLogStartInd, GazeLogEndInd)
        
        if GazeLogStartInd != GazeLogEndInd:
            gazeEpochL = pupilLogL[GazeLogStartInd:GazeLogEndInd]
            gazeEpochR = pupilLogR[GazeLogStartInd:GazeLogEndInd]
            interPupilDist = pupilDist[GazeLogStartInd:GazeLogEndInd]
            
            winSize = 25
            # Filter pupil sizes
            pupilWoOutlierL = hampel(gazeEpochL, winSize, 3)
            pupilWoOutlierR = hampel(gazeEpochR, winSize, 3)
            
            # Does pupilWoOutlier still have nan values -- happens in subject 'lr', the complete list is made of nan values
            if pupilWoOutlierL.isnull().values.any() == True or pupilWoOutlierR.isnull().values.any() == True:
                # This means that the complete list was already only nan values, and cannot be changed, whatsoever
                continue 
            
            # compute correlation between right and left eye pupil sizes
            pupilCorr = pupilWoOutlierL.corrwith(pupilWoOutlierR, axis = 0)
            if pupilCorr.values[0] < 0.8: # 0.8 is a good value for correlation and was also the mean of mean of the correlations
            # for the trials for the users
                continue
            
            #print(subjName, len(pupilWoOutlierL), len(pupilAbsoluteL))             
                
            # Moving Mean of data without outliers: 
            pupilAbsoluteL = pupilWoOutlierL.rolling(window=winSize, min_periods=1, center=True).mean()
            pupilAbsoluteR = pupilWoOutlierR.rolling(window=winSize, min_periods=1, center=True).mean()
            
            # Reference of 400ms, and not only the first observation point
            pupilReferenceL = pupilAbsoluteL[0][0:39].mean()
            pupilReferenceR = pupilAbsoluteR[0][0:39].mean()
            
            pupilRelativeL = (pupilAbsoluteL - pupilReferenceL)/pupilReferenceL
            pupilRelativeR = (pupilAbsoluteR - pupilReferenceR)/pupilReferenceR
            
        else:
            print('one')
            continue
            #continue # remove samples with only 1 sample
        
        #print(pupilRelativeL)
        # Also to check if phrase typed is correct or not, only check the last element of epoch
        #print(timeList[-1])

        timeOfPhraseScratchedNow, phraseScratchedInd = nearestTimePoint(timeScratchAll, timeList)
        timeOfPhraseToBeScratchedNow, phraseToBeScratchedInd = nearestTimePoint(timeToBeScratchedAll, timeList)
        
        if phraseScratchedInd < 0:
            phraseScratched = ''
        else:
            phraseScratched = scratchPadList[phraseScratchedInd][1]
        
        phraseToBeScratched = phraseList[phraseToBeScratchedInd][1]
        
        #print(phraseScratched, phraseToBeScratched)
        
    
            # are they the same? yes/no -> save in list with the time
        if any(char.isdigit() for char in phraseToBeScratched) and phraseScratched in phraseToBeScratched:
            # For correct entries
            if 'THE EXPERIMENT IS NOW DONE' in phraseToBeScratched:
                #print('exp done')
                continue
            # add the pupil size
            #print('CORRECTLY WRITTEN', phraseScratched, 'FROM', phraseToBeScratched)
            pupilData = appendDictionary(pupilData, 'Correct', pupilAbsoluteL, pupilAbsoluteR, pupilRelativeL, pupilRelativeR, absoluteSize)
                
        elif not (any(char.isdigit() for char in phraseToBeScratched)) and phraseToBeScratched in phraseScratched:
            # For correct entries
            if 'THE EXPERIMENT IS NOW DONE' in phraseToBeScratched:
                #print('exp done')
                continue
            pupilData = appendDictionary(pupilData, 'Correct', pupilAbsoluteL, pupilAbsoluteR, pupilRelativeL, pupilRelativeR, absoluteSize)
            
        else:
            pupilData = appendDictionary(pupilData, 'Incorrect', pupilAbsoluteL, pupilAbsoluteR, pupilRelativeL, pupilRelativeR, absoluteSize)

    return pupilData

In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['CorrectFirst'] = 0
pupilData['IncorrectFirst'] = 0
pupilData['CorrectNumber'] = 0
pupilData['IncorrectNumber'] = 0

for root, dirs, files in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in files:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                
                continue
            else:
                
                # Compute dwell time
                userKeysWithTime = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                scratchPadKeyTime = UserLookedAtScratchPad(userKeysWithTime)
                
                # for every element, find the time closest and previous to it, and check what was typed 
                # AND what should have been typed
                # Also, add gaze data to epoch, but first subtract the fixation time and complete the pupil data filtering as
                # per Per's paper
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                pupilData = computeAggregateAverage(scratchPadKeyTime, scratchPad, stimPhrase, gazeLog, pupilData, 800, subjName)
                #print(pupilData)



In [ ]:
if pupilData['CorrectFirst'] > 0 or pupilData['IncorrectFirst'] > 0:
    #print(pupilData)
    # if the Correct and Incorrect data are of different sizes,
    pupilSizeMin = min(len(pupilData['CorrectLeft']), len(pupilData['IncorrectLeft']))
    fig = plt.figure()
    axL = fig.add_subplot(2, 1, 1)
    axR = fig.add_subplot(2, 1, 2)
    xAxis = np.arange(0, float(pupilSizeMin/90), float(1/90))
    pupilPlot = dict()
    pupilPlot['CorrectLeft'] = [x/pupilData['CorrectNumber'] for x in pupilData['CorrectLeft']]
    pupilPlot['CorrectRight'] = [x/pupilData['CorrectNumber'] for x in pupilData['CorrectRight']]
    pupilPlot['IncorrectLeft'] = [x/pupilData['IncorrectNumber'] for x in pupilData['IncorrectLeft']]
    pupilPlot['IncorrectRight'] = [x/pupilData['IncorrectNumber'] for x in pupilData['IncorrectRight']]
    
    # Standard deviation
#     pupilPlot['CorrectLeftSquareRoot'] = [(math.fabs(pupilData['CorrectLeftSquared'][xInd]/pupilData['CorrectNumber']-(pupilPlot['CorrectLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['CorrectLeftSquared']))]
#     pupilPlot['CorrectRightSquareRoot'] = [(math.fabs(pupilData['CorrectRightSquared'][xInd]/pupilData['CorrectNumber']-(pupilPlot['CorrectRight'][xInd]))**2)**0.5 for xInd in range(0, len(pupilData['CorrectRightSquared']))]
#     pupilPlot['IncorrectLeftSquareRoot'] = [(math.fabs(pupilData['IncorrectLeftSquared'][xInd]/pupilData['IncorrectNumber']-(pupilPlot['IncorrectLeft'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['IncorrectLeftSquared']))]
#     pupilPlot['IncorrectRightSquareRoot'] = [(math.fabs(pupilData['IncorrectRightSquared'][xInd]/pupilData['IncorrectNumber']-(pupilPlot['IncorrectRight'][xInd])**2))**0.5 for xInd in range(0, len(pupilData['IncorrectRightSquared']))]

    pupilPlot['CorrectLeftSquareRoot'] = [(math.fabs(((pupilData['CorrectNumber'])*(pupilData['CorrectLeftSquared'][xInd])-(pupilData['CorrectLeft'][xInd])**2)/((pupilData['CorrectNumber'])*(pupilData['CorrectNumber']-1))))**0.5 for xInd in range(0, len(pupilData['CorrectLeftSquared']))]
    pupilPlot['CorrectRightSquareRoot'] = [(math.fabs(((pupilData['CorrectNumber'])*(pupilData['CorrectRightSquared'][xInd])-(pupilData['CorrectRight'][xInd])**2)/((pupilData['CorrectNumber'])*(pupilData['CorrectNumber']-1))))**0.5 for xInd in range(0, len(pupilData['CorrectRightSquared']))]
    pupilPlot['IncorrectLeftSquareRoot'] = [(math.fabs(((pupilData['IncorrectNumber'])*(pupilData['IncorrectLeftSquared'][xInd])-(pupilData['IncorrectLeft'][xInd])**2)/((pupilData['IncorrectNumber'])*(pupilData['IncorrectNumber']-1))))**0.5 for xInd in range(0, len(pupilData['IncorrectLeftSquared']))]
    pupilPlot['IncorrectRightSquareRoot'] = [(math.fabs(((pupilData['IncorrectNumber'])*(pupilData['IncorrectRightSquared'][xInd])-(pupilData['IncorrectRight'][xInd])**2)/((pupilData['IncorrectNumber'])*(pupilData['IncorrectNumber']-1))))**0.5 for xInd in range(0, len(pupilData['IncorrectRightSquared']))]

    pupilPlot['CorrectLeftStdPositive'] = [pupilPlot['CorrectLeft'][xInd] + pupilPlot['CorrectLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['CorrectLeft']))]
    pupilPlot['CorrectLeftStdNegative'] = [pupilPlot['CorrectLeft'][xInd] - pupilPlot['CorrectLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['CorrectLeft']))]
    pupilPlot['IncorrectLeftStdPositive'] = [pupilPlot['IncorrectLeft'][xInd] + pupilPlot['IncorrectLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['IncorrectLeft']))]
    pupilPlot['IncorrectLeftStdNegative'] = [pupilPlot['IncorrectLeft'][xInd] - pupilPlot['IncorrectLeftSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['IncorrectLeft']))]

    pupilPlot['CorrectRightStdPositive'] = [pupilPlot['CorrectRight'][xInd] + pupilPlot['CorrectRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['CorrectRight']))]
    pupilPlot['CorrectRightStdNegative'] = [pupilPlot['CorrectRight'][xInd] - pupilPlot['CorrectRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['CorrectRight']))]
    pupilPlot['IncorrectRightStdPositive'] = [pupilPlot['IncorrectRight'][xInd] + pupilPlot['IncorrectRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['IncorrectRight']))]
    pupilPlot['IncorrectRightStdNegative'] = [pupilPlot['IncorrectRight'][xInd] - pupilPlot['IncorrectRightSquareRoot'][xInd] for xInd in range(0, len(pupilPlot['IncorrectRight']))]


    axL.plot(xAxis, pupilPlot['CorrectLeft'][0:pupilSizeMin], 'bo', label = 'Correct')
    axL.plot(xAxis, pupilPlot['IncorrectLeft'][0:pupilSizeMin], 'ro', label = 'Incorrect')
    axL.fill_between(xAxis, pupilPlot['CorrectLeftStdPositive'][0:pupilSizeMin],  pupilPlot['CorrectLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axL.fill_between(xAxis, pupilPlot['IncorrectLeftStdPositive'][0:pupilSizeMin], pupilPlot['IncorrectLeftStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')

    
    axL.set_title('Left pupil')
    axL.set_xlabel('Time [in s]')
    if absoluteSize:
        axL.set_ylabel('Absolute pupil size [in mm]')
    else:
        axL.set_ylabel('Relative pupil size')
    axL.legend()

    axR.plot(xAxis, pupilPlot['CorrectRight'][0:pupilSizeMin], 'bo', label = 'Correct')
    axR.plot(xAxis, pupilPlot['IncorrectRight'][0:pupilSizeMin], 'ro', label = 'Incorrect')
    axR.fill_between(xAxis, pupilPlot['CorrectRightStdPositive'][0:pupilSizeMin], pupilPlot['CorrectRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'blue')
    axR.fill_between(xAxis, pupilPlot['IncorrectRightStdPositive'][0:pupilSizeMin], pupilPlot['IncorrectRightStdNegative'][0:pupilSizeMin], alpha = 0.15, color = 'red')
    
    axR.set_title('Right pupil')
    axR.legend()
    axR.set_xlabel('Time [in s]')
    if absoluteSize:
        axR.set_ylabel('Absolute pupil size [in mm]')
    else:
        axR.set_ylabel('Relative pupil size')


In [ ]:
pupilData['CorrectNumber']

In [ ]:
pupilData['IncorrectNumber']

In [ ]:
# plot again, if required

pupilSizeMin = min(len(pupilData['CorrectLeft']), len(pupilData['IncorrectLeft']))
fig = plt.figure()
axL = fig.add_subplot(2, 1, 1)
axR = fig.add_subplot(2, 1, 2)
xAxis = np.arange(0, float(pupilSizeMin/90), float(1/90))
pupilPlot = dict()
pupilPlot['CorrectLeft'] = [x/pupilData['CorrectNumber'] for x in pupilData['CorrectLeft']]
pupilPlot['CorrectRight'] = [x/pupilData['CorrectNumber'] for x in pupilData['CorrectRight']]
pupilPlot['IncorrectLeft'] = [x/pupilData['IncorrectNumber'] for x in pupilData['IncorrectLeft']]
pupilPlot['IncorrectRight'] = [x/pupilData['IncorrectNumber'] for x in pupilData['IncorrectRight']]

# Standard deviation
pupilPlot['CorrectLeftSquared'] = [pupilData['CorrectLeftSquared'][xInd]/pupilData['CorrectNumber']-pupilPlot['CorrectLeft'][xInd] for xInd in range(0, len(pupilData['CorrectLeftSquared']))]
pupilPlot['CorrectRightSquared'] = [pupilData['CorrectRightSquared'][xInd]/pupilData['CorrectNumber']-pupilPlot['CorrectRight'][xInd] for xInd in range(0, len(pupilData['CorrectRightSquared']))]
pupilPlot['IncorrectLeftSquared'] = [pupilData['IncorrectLeftSquared'][xInd]/pupilData['IncorrectNumber']-pupilPlot['IncorrectLeft'][xInd] for xInd in range(0, len(pupilData['IncorrectLeftSquared']))]
pupilPlot['IncorrectRightSquared'] = [pupilData['IncorrectRightSquared'][xInd]/pupilData['IncorrectNumber']-pupilPlot['IncorrectRight'][xInd] for xInd in range(0, len(pupilData['IncorrectRightSquared']))]

pupilPlot['CorrectLeftStdPositive'] = [pupilPlot['CorrectLeft'][xInd] + pupilPlot['CorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectLeft']))]
pupilPlot['CorrectLeftStdNegative'] = [pupilPlot['CorrectLeft'][xInd] - pupilPlot['CorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectLeft']))]
pupilPlot['IncorrectLeftStdPositive'] = [pupilPlot['IncorrectLeft'][xInd] + pupilPlot['IncorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectLeft']))]
pupilPlot['IncorrectLeftStdNegative'] = [pupilPlot['IncorrectLeft'][xInd] - pupilPlot['IncorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectLeft']))]

pupilPlot['CorrectRightStdPositive'] = [pupilPlot['CorrectRight'][xInd] + pupilPlot['CorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectRight']))]
pupilPlot['CorrectRightStdNegative'] = [pupilPlot['CorrectRight'][xInd] - pupilPlot['CorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectRight']))]
pupilPlot['IncorrectRightStdPositive'] = [pupilPlot['IncorrectRight'][xInd] + pupilPlot['IncorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectRight']))]
pupilPlot['IncorrectRightStdNegative'] = [pupilPlot['IncorrectRight'][xInd] - pupilPlot['IncorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectRight']))]


axL.plot(xAxis, pupilPlot['CorrectLeft'][0:pupilSizeMin], 'bo', label = 'Correct')
axL.plot(xAxis, pupilPlot['IncorrectLeft'][0:pupilSizeMin], 'ro', label = 'Incorrect')
axL.plot(xAxis, pupilPlot['CorrectLeftStdPositive'][0:pupilSizeMin], ':', color = 'blue')
axL.plot(xAxis, pupilPlot['CorrectLeftStdNegative'][0:pupilSizeMin], ':', color = 'blue')
axL.plot(xAxis, pupilPlot['IncorrectLeftStdPositive'][0:pupilSizeMin], ':', color = 'red')
axL.plot(xAxis, pupilPlot['IncorrectLeftStdNegative'][0:pupilSizeMin], ':', color = 'red')
    
axL.set_title('Left')
axL.set_xlabel('Time [in s]')
if absoluteSize:
    axL.set_ylabel('Absolute pupil size [in mm]')
else:
    axL.set_ylabel('Relative pupil size [in %]')
axL.legend()

axR.plot(xAxis, pupilPlot['CorrectRight'][0:pupilSizeMin], 'bo', label = 'Correct')
axR.plot(xAxis, pupilPlot['IncorrectRight'][0:pupilSizeMin], 'ro', label = 'Incorrect')
axR.plot(xAxis, pupilPlot['CorrectRightStdPositive'][0:pupilSizeMin], ':', color = 'blue')
axR.plot(xAxis, pupilPlot['CorrectRightStdNegative'][0:pupilSizeMin], ':', color = 'blue')
axR.plot(xAxis, pupilPlot['IncorrectRightStdPositive'][0:pupilSizeMin], ':', color = 'red')
axR.plot(xAxis, pupilPlot['IncorrectRightStdNegative'][0:pupilSizeMin], ':', color = 'red')

axR.legend()
axR.set_xlabel('Time [in s]')
if absoluteSize:
    axR.set_ylabel('Absolute pupil size [in mm]')
else:
    axR.set_ylabel('Relative pupil size [in %]')
